2022-02-02 21:18:45,718 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-02-02 21:18:45,776 - root - INFO - ** Kedro project time_series_kedro
2022-02-02 21:18:45,777 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


In [9]:
%reload_kedro
train = catalog.load("train_data")
prepared_data = catalog.load("prepared_data")
test = catalog.load("test_data")
transactions = catalog.load("transactions")

2022-02-02 21:35:22,091 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-02-02 21:35:22,129 - root - INFO - ** Kedro project time_series_kedro
2022-02-02 21:35:22,130 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`
2022-02-02 21:35:22,131 - kedro.io.data_catalog - INFO - Loading data from `train_data` (CSVDataSet)...
2022-02-02 21:35:23,594 - kedro.io.data_catalog - INFO - Loading data from `prepared_data` (CSVDataSet)...
2022-02-02 21:35:23,623 - kedro.io.data_catalog - INFO - Loading data from `test_data` (CSVDataSet)...
2022-02-02 21:35:23,961 - kedro.io.data_catalog - INFO - Loading data from `transactions` (CSVDataSet)...


In [5]:
import pandas as pd
import numpy as np

In [10]:
prepared_data

,date,family,sales
0,2013-01-01,AUTOMOTIVE,0.000000
1,2013-01-01,BABY CARE,0.000000
2,2013-01-01,BEAUTY,2.000000
3,2013-01-01,BEVERAGES,810.000000
4,2013-01-01,BOOKS,0.000000
...,...,...,...
55567,2017-08-15,POULTRY,17586.709986
55568,2017-08-15,PREPARED FOODS,4641.522980
55569,2017-08-15,PRODUCE,125108.971000
55570,2017-08-15,SCHOOL AND OFFICE SUPPLIES,2530.000000


In [37]:
from time_series_kedro.extras.utils import rolling_fill
def build_series(serie_data, serie_target, date_col):
    serie = serie_data.set_index(date_col)[[serie_target]]
    full_serie = serie.reindex(pd.date_range(serie.index.min(), serie.index.max()))
    full_serie[serie_target] = rolling_fill(full_serie[serie_target], n=2)
    return full_serie
    
    

In [39]:
serie_target = "sales"
date_col = "date"
prepared_data.groupby("family").apply(lambda serie_data: build_series(serie_data, serie_target, date_col))

sales    0
dtype: int64

In [12]:
serie_data = prepared_data[prepared_data.family=="AUTOMOTIVE"].set_index("date")
serie_data

,family,sales
date,,
2013-01-01,AUTOMOTIVE,0.0
2013-01-02,AUTOMOTIVE,255.0
2013-01-03,AUTOMOTIVE,161.0
2013-01-04,AUTOMOTIVE,169.0
2013-01-05,AUTOMOTIVE,342.0
...,...,...
2017-08-11,AUTOMOTIVE,441.0
2017-08-12,AUTOMOTIVE,403.0
2017-08-13,AUTOMOTIVE,481.0


In [18]:
x = pd.DataFrame(data={"v":[1, 2, 3, 4, 5]}, index=[1, 3, 4, 5,7])
x.reindex([1, 2, 3, 4, 5, 6, 7])

,v
1,1
2,2
3,2
4,3
5,4
6,5
7,5


In [22]:
serie_data = prepared_data[prepared_data.family=="AUTOMOTIVE"].set_index("date")
serie_data = serie_data.reindex(pd.date_range(serie_data.index.min(), serie_data.index.max()))
serie_data[serie_id] = serie_data[serie_id].fillna("ffill")


In [6]:
pd.to_datetime(train.date)

id
0         2013-01-01
1         2013-01-01
2         2013-01-01
3         2013-01-01
4         2013-01-01
             ...    
3000883   2017-08-15
3000884   2017-08-15
3000885   2017-08-15
3000886   2017-08-15
3000887   2017-08-15
Name: date, Length: 3000888, dtype: datetime64[ns]

In [5]:
train.groupby("family").nunique().date

family
AUTOMOTIVE                    1684
BABY CARE                     1684
BEAUTY                        1684
BEVERAGES                     1684
BOOKS                         1684
BREAD/BAKERY                  1684
CELEBRATION                   1684
CLEANING                      1684
DAIRY                         1684
DELI                          1684
EGGS                          1684
FROZEN FOODS                  1684
GROCERY I                     1684
GROCERY II                    1684
HARDWARE                      1684
HOME AND KITCHEN I            1684
HOME AND KITCHEN II           1684
HOME APPLIANCES               1684
HOME CARE                     1684
LADIESWEAR                    1684
LAWN AND GARDEN               1684
LINGERIE                      1684
LIQUOR,WINE,BEER              1684
MAGAZINES                     1684
MEATS                         1684
PERSONAL CARE                 1684
PET SUPPLIES                  1684
PLAYERS AND ELECTRONICS       1684
POULTRY      